In [1]:
import numpy as np
import json

# Character 2 Ordinal Encoding

In [2]:
# Read Character to Ordinal Encoding Mapping
# with open('/kaggle/input/asl-fingerspelling/character_to_prediction_index.json') as json_file:
#     CHAR2ORD = json.load(json_file)

CHAR2ORD = {
    " ":0, "!":1, "#":2, "$":3, "%":4, "&":5, "'":6, "(":7, ")":8, "*":9,
    "+":10, ",":11, "-":12, ".":13, "\/":14, "0":15, "1":16, "2":17, "3":18, "4":19,
    "5":20, "6":21, "7":22, "8":23, "9":24, ":":25, ";":26, "=":27, "?":28, "@":29,
    "[":30, "_":31, "a":32, "b":33, "c":34, "d":35, "e":36, "f":37, "g":38, "h":39,
    "i":40, "j":41, "k":42, "l":43, "m":44, "n":45, "o":46, "p":47, "q":48, "r":49,
    "s":50, "t":51, "u":52, "v":53, "w":54, "x":55, "y":56, "z":57, "~":58
}

# Ordinal to Character Mapping
ORD2CHAR = {j:i for i, j in CHAR2ORD.items()}
    
# Character to Ordinal Encoding Mapping   
# display(pd.Series(CHAR2ORD).to_frame('Ordinal Encoding'))

# Global Config

In [3]:
# Length of Phrase + EOS Token
MAX_PHRASE_LENGTH = 31 + 1

# Load X/y

In [4]:
# TRAIN
X = np.load('/kaggle/input/aslfr-preprocessing-dataset/X.npy')
y = np.load('/kaggle/input/aslfr-preprocessing-dataset/y.npy')[:, :MAX_PHRASE_LENGTH]
# N_TRAIN_SAMPLES = len(X_train)
# print(f'X_train shape: {X_train.shape}')

In [5]:
X = X[:, :, 0:84]

In [6]:
def rotate_point(x, y, angle_in_degrees):
    theta = np.radians(angle_in_degrees)
    cos_theta, sin_theta = np.cos(theta), np.sin(theta)
    x_prime = x * cos_theta + y * sin_theta
    y_prime = -x * sin_theta + y * cos_theta
    return x_prime, y_prime

def rotate_vector(vector, angle_in_degrees):
    # split the vector into three parts: x and y coordinates for the left hand, right hand, and face
    x_left_hand = vector[0:21]
    y_left_hand = vector[21:42]
    x_right_hand = vector[42:63]
    y_right_hand = vector[63:84]
    
    new_vector = []
    for x_coordinates, y_coordinates in [(x_left_hand, y_left_hand), (x_right_hand, y_right_hand)]:
        for x, y in zip(x_coordinates, y_coordinates):
            new_x, new_y = rotate_point(x, y, angle_in_degrees)
            new_vector.append(new_x)
            new_vector.append(new_y)
    
    return new_vector

In [7]:
# intialise augmented datasets
X1, X2 = X, X

In [8]:
X_shape = X.shape

# process augmented dataset 1
for i in range(X_shape[0]):
    for j in range(X_shape[1]):
        X1[i, j] = rotate_vector(X1[i, j], 2)

# process augmented dataset 2
for i in range(X_shape[0]):
    for j in range(X_shape[1]):
        X2[i, j] = rotate_vector(X2[i, j], -2)

In [9]:
# concatenate original and augmented datasets
X = np.concatenate((X, X1, X2), axis=2)
y = np.concatenate((y, y, y), axis=1)

In [10]:
# save X and y
np.save('X.npy', X)
np.save('y.npy', y)